<h3>Intraday Strategy Using GARCH Model.</h3>
<ul>
    <li>Using simulated daily data and intraday 5-min data.</li>
    <li>Load Daily and 5-minute data.</li>
    <li>Define funtion to fit GARCH model on the daily data and predict 1-day ahead volatility in a rolling window.</li>
    <li>Calculate predication premium and form a daily signla from it.</li>
    <li>Merge with intraday data and calculate intraday indicators to form the intraday signal.</li>
    <li>Generate the position entry and hold until the end of the day.</li>
    <li>Calculate final strategy returns.</li>
</ul>

<h3>1. Load Simulated Daily and Simulated 5-minute data.</h3>
<ul>
    <li>We are loading both datasets, set the indexes and calculate daily log returns.</li>
</ul>

In [15]:
import matplotlib.pyplot as plt
from arch import arch_model
import pandas as pd
import numpy as np
import os

# Define the path to the folder containing the data files.
# This path will be used to load CSV files containing daily and intraday data.
# data_folder = '/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/data'

# Load the simulated daily data from a CSV file into a pandas DataFrame.
# The CSV file is expected to contain columns like 'Date' and 'Adj Close'.
# daily_df = pd.read_csv(os.path.join(data_folder, 'simulated_daily_data.csv'))

# Convert the 'Date' column in the daily data to datetime format.
# This is essential for time-series analysis and setting the date as an index.
# daily_df["Date"]= pd.to_datetime(daily_df['Date'])

# Set the 'Date' column as the index of the DataFrame.
# This allows for easier time-series operations, such as resampling or filtering by date.
# daily_df = daily_df.set_index('Date')

# Calculate the logarithmic returns based on the 'Adj Close' prices.
# The log returns are computed by taking the natural log of the adjusted close price and then differencing it.
# This is a common transformation used in financial analysis to analyze returns.
# daily_df['log_ret'] = np.log(daily_df['Adj Close']).diff()

# Load the simulated intraday 5-minute data from a CSV file into a pandas DataFrame.
# The CSV file is expected to contain columns like 'datetime' and other relevant financial data.
# intraday_5min_df = pd.read_csv(os.path.join(data_folder, 'simulated_5min_data.csv'))

# Convert the 'datetime' column in the intraday data to datetime format.
# This ensures that the timestamp is correctly interpreted for time-series analysis.
# intraday_5min_df['datetime'] = pd.to_datetime(intraday_5min_df['datetime'])

# Set the 'datetime' column as the index of the DataFrame.
# This allows for easier time-series operations, such as filtering or aggregating data by time intervals.
# intraday_5min_df = intraday_5min_df.set_index('datetime')
# intraday_5min_df


# Convert the 'Unnamed: 6' column (which likely represents an additional timestamp or date) to datetime format.
# This step is crucial if this column is to be used for further time-based operations.
# intraday_5min_df['Unnamed: 6'] = pd.to_datetime(intraday_5min_df['Unnamed: 6']) 

# Extract only the date component from the index of the intraday data.
# This could be useful for aligning intraday data with daily data or performing date-based groupings.
# intraday_5min_df.index.date

array([datetime.date(2021, 9, 29), datetime.date(2021, 9, 29),
       datetime.date(2021, 9, 29), ..., datetime.date(2023, 9, 20),
       datetime.date(2023, 9, 20), datetime.date(2023, 9, 20)],
      dtype=object)

<h3>2. Define function to fit GARCH model and predict 1-day ahead volatility in a rolling window. </h3>
<ul>
    <li>We are first calculating the 6-month rolling variance and then we are creating a function in a 6-month rolling window to fit a garch model and predict the next day variance.</li>
</ul>

In [22]:
# To create a fit to the GARCH model. Autoregressive orders. BruteForce fit 16 models combination 1,2,3,4 16 models extract Mean Squared error and Bayseian Information Criteria.
daily_df['variance'] = daily_df['log_ret'].rolling(180).var()

daily_df = daily_df['2020-01-01':].copy()


def predict_volatility(x):
    
    best_model = arch_model(y=x,
                            p=1,
                            q=3).fit(update_freq=5, disp='off')

    variance_forecast = best_model.forecast(horizon=1).variance.iloc[-1, 0]

    print(x.index[-1])


    return variance_forecast


daily_df['predictions'] = daily_df['log_ret'].rolling(180).apply(lambda x: predict_volatility(x)) 


daily_df

/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002459. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002459. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This war

2020-06-28 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-07-01 00:00:00
2020-07-02 00:00:00
2020-07-03 00:00:00
2020-07-04 00:00:00


/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.00243. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002417. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warn

2020-07-05 00:00:00
2020-07-06 00:00:00
2020-07-07 00:00:00
2020-07-08 00:00:00
2020-07-09 00:00:00
2020-07-10 00:00:00
2020-07-11 00:00:00
2020-07-12 00:00:00
2020-07-13 00:00:00
2020-07-14 00:00:00


/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002377. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002377. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This war

...

2023-08-30 00:00:00
2023-08-31 00:00:00
2023-09-01 00:00:00
2023-09-02 00:00:00
2023-09-03 00:00:00
2023-09-04 00:00:00
2023-09-05 00:00:00
2023-09-06 00:00:00
2023-09-07 00:00:00
2023-09-08 00:00:00
2023-09-09 00:00:00
2023-09-10 00:00:00


/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0005066. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.000506. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This 

2023-09-11 00:00:00
2023-09-12 00:00:00
2023-09-13 00:00:00
2023-09-14 00:00:00
2023-09-15 00:00:00
2023-09-16 00:00:00
2023-09-17 00:00:00
2023-09-18 00:00:00


/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003823. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/codymckeon/Library/Mobile Documents/com~apple~CloudDocs/ml_projects/unsupervised/intraday_garch_model/venv/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003811. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This

,Open,High,Low,Close,Adj Close,Volume,Unnamed: 7,log_ret,variance,predictions
Date,,,,,,,,,,
2020-01-01,1798.723022,1813.582642,1793.736084,1800.043579,1800.043579,1.856566e+10,NaN,0.000914,NaN,NaN
2020-01-02,1800.637817,1803.038818,1733.817505,1746.367554,1746.367554,2.080208e+10,NaN,-0.030273,NaN,NaN
2020-01-03,1746.107178,1853.428833,1728.749023,1836.221069,1836.221069,2.811148e+10,NaN,0.050172,NaN,NaN
2020-01-04,1836.343872,1856.846436,1827.378540,1852.664185,1852.664185,1.844427e+10,NaN,0.008915,NaN,NaN
2020-01-05,1852.612915,1886.124268,1850.133911,1852.829346,1852.829346,1.972507e+10,NaN,0.000089,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-09-14,6557.069336,6693.655762,6542.862793,6634.918457,6634.918457,1.381136e+10,NaN,0.011801,0.000393,0.000334
2023-09-15,6633.454590,6710.124512,6560.175293,6652.173340,6652.173340,1.147974e+10,NaN,0.002597,0.000385,0.000292
2023-09-16,6651.549805,6688.692383,6618.472656,6642.070313,6642.070313,7.402031e+09,NaN,-0.001520,0.000384,0.000286
